### Idiomatic Python: Using a `deque`

Far too often I see Python code that inserts or deletes elements at the **front** of a list.

For example code that looks like this:

In [1]:
l = [1, 2, 3, 4, 5]

In [2]:
l.insert(0, "new element")
l

['new element', 1, 2, 3, 4, 5]

or

In [3]:
del l[0]
l

[1, 2, 3, 4, 5]

or

In [4]:
l.pop(0)

1

The problem with inserting or deleting elements at the beginning of a list is that it is very inefficient. Sure, if you do in a few times here and there in your app, there is no need to worry about it - but if you find these methods being used repeatedly in your program, you will see definite performance degradation. And the larger the list, the worse it gets.

Appending or removing the last element of a list is not an issue, just from the front.

Let's timings for both of these:

In [5]:
from timeit import timeit

In [6]:
l = list(range(100_000))
timeit("l.insert(0,0)", globals=globals(), number=100_000)

7.414582791971043

In [7]:
l = list(range(100_000))
timeit("l.append(0)", globals=globals(), number=100_000)

0.003216374898329377

In [8]:
l = list(range(100_000))
timeit("del l[0]", globals=globals(), number=100_000)

0.7904178330209106

In [9]:
l = list(range(100_000))
timeit("del l[-1]", globals=globals(), number=100_000)

0.0028052921406924725

As you can see the difference is significant!

If you find yourself needing to perform those types of operations, then the correct data structure to use is the `deque`, found in the `collections` module.

[https://docs.python.org/3/library/collections.html#collections.deque](https://docs.python.org/3/library/collections.html#collections.deque)

In [10]:
from collections import deque

A deque is a very efficient list-like structure for adding/removing elements from either side of it.

Technically, deleting and inserting elements at the start ot a list has **O(n)** complexity, whereas the deque as **O(1)** complexity.

By default deques have arbitrary size (just like lists), but can also be declared to be of fixed size.

Once a fixed-size deque fills up, any append operation on one end will push an element out of the other end.

A deque is essentially a double-ended queue (hence the name deque).

We can use indexing to access elements directly in the deque, but be aware that this access can be slower than accessing elements by index in a list (especially in the middle of the deque). Accessing the element at the beginning or the end of a deque however is fast.

So there are some tradeoffs, but if you find using a list for queue-like behavior, inserting and deleting elements at the beginning of the list, then a deque is usually the better choice.

#### Infinite Deques

Let's look at an infinite deque first.

We can create an empty deque, or initialize it with some values by passing an iterable to its constructor.

In [11]:
dq = deque([1, 2, 3, 4, 5])
dq

deque([1, 2, 3, 4, 5])

And we can access elements by index:

In [12]:
dq[0], dq[1]

(1, 2)

We can append to the left or the right of the deque:

In [13]:
dq.append(6)
dq

deque([1, 2, 3, 4, 5, 6])

In [14]:
dq.appendleft(0)
dq

deque([0, 1, 2, 3, 4, 5, 6])

We can pop elements from the left or the right:

In [15]:
dq.popleft()
dq

deque([1, 2, 3, 4, 5, 6])

In [16]:
dq.pop()
dq

deque([1, 2, 3, 4, 5])

We can also extend a deque (just like lists), but we can extend either at the left or the right of the deque:

In [17]:
dq.extend(['y', 'z'])
dq

deque([1, 2, 3, 4, 5, 'y', 'z'])

In [18]:
dq.extendleft(['a', 'b'])
dq

deque(['b', 'a', 1, 2, 3, 4, 5, 'y', 'z'])

> Note how the `extendleft` process the iterable we are extending the deque with - each element is added to the left of the list, so the final ordering of the elements in the deque is in reverse order from the order of the elements in the iterable

You can get the number of elements in a deque using the usual `len()` function:

In [19]:
len(dq)

9

Deques have many other functions available, so check out the Python docs in the link I provided above.

#### Finite Deques

Next, let's look at bounded deques - the methods available are the same, but the behavior is slightly different because the deque now has a max size, after which inserting elements at the left/right of the deque will cause elements to drop off from the other end (so very much like a bounded queue, except the dequeue works in both directions).

In [20]:
dq = deque([1, 2, 3, 4, 5], maxlen=5)
dq

deque([1, 2, 3, 4, 5], maxlen=5)

In [21]:
dq.appendleft(0)
dq

deque([0, 1, 2, 3, 4], maxlen=5)

As you can see, the element `5` dropped off the right of the deque.

In [22]:
dq.append(5)
dq

deque([1, 2, 3, 4, 5], maxlen=5)

As you can see, the first element dropped off when we added `5` to the right of the deque.

#### Timings

Let's look at some timings to compare the operations on the left/right ends of a deque and a list.

In [23]:
l = list(range(100_000))
timeit("l.insert(0,0)", globals=globals(), number=10_000)

0.5150752498302609

In [24]:
dq = deque(range(100_000))
timeit("dq.appendleft(0)", globals=globals(), number=10_000)

0.00041258311830461025

As you can see, a dequeue is substantially better for inserting at the left than a list.

For appending to the right, there is no difference:

In [25]:
l = list(range(100_000))
timeit("l.append(0)", globals=globals(), number=10_000)

0.0003307501319795847

In [26]:
dq = deque(range(100_000))
timeit("dq.append(0)", globals=globals(), number=10_000)

0.00040837517008185387

And same happens with deleting an element from the left and the right:

In [27]:
l = list(range(100_000))
timeit("l.pop(0)", globals=globals(), number=10_000)

0.15366895799525082

In [28]:
dq = deque(range(100_000))
timeit("dq.popleft()", globals=globals(), number=10_000)

0.0004180839750915766

In [29]:
l = list(range(100_000))
timeit("l.pop(-1)", globals=globals(), number=10_000)

0.00037158397026360035

In [30]:
dq = deque(range(100_000))
timeit("dq.pop()", globals=globals(), number=10_000)

0.0004200420808047056

#### Conclusion

The takeaway here is to always use the most appropriate data structure for your particular circumstance. And if you need to continuously insert/delete elements from the left of a list, you should really look at using a deque instead - the performance improvements can be substantial.

If you find that you really need slicing or direct access to elements inside the deque (not the left or right elements), then you should careful time your code and see which structure will perform better. Or you could even have hybrid approaches, where you can perform the insertions/deletions on your deque in a first phase, then, assuming the dequeue is not stable, extract the dequeue elements as a list, and in a second phase do your slicing and index lookups.

Getting a list from deque elements is simple - since a dequeu is an iterable is you can simply pass the deque to the `list()` constructor:

In [31]:
dq = deque([1, 2, 3, 4, 5])
l = list(dq)
type(l), l

(list, [1, 2, 3, 4, 5])